                                                 Project 2 
                                     Advanced Database and Information Systems
                                                  Submited by:
                                         Anh-My Buis   ,    Shaun Saju

PART A

In [2]:
import pandas as pd
from collections import defaultdict

In [2]:
# Opening the file
df1 = pd.read_csv("100k.txt", header=None)

In [3]:
# Split by dataframe into 3 Columns, SUBJECT, PREDICATE, OBJECT i.e the TRIPLE STORE
df1 = df1[0].str.split('\t', expand=True).rename(columns = {0:'SUBJECT',1:'ATTRIBUTE',2:'OBJECT'})

In [4]:
# Removing ". " at the end 
df1['OBJECT'] = df1['OBJECT'].str[:-2] 

In [5]:
# Vertical Partitioning the triple store by predicate
groups = df1.groupby('ATTRIBUTE')[['SUBJECT','OBJECT']]

In [6]:
# Getting the relations we are interested in.
dfs = {}
for attribute in ['follows','friendOf', 'likes', 'hasReview']:
    dfs[attribute] =  groups.get_group(df1[df1['ATTRIBUTE'].str.contains(attribute)]['ATTRIBUTE'].unique()[0])

PART B

In [7]:
# Hash function
def rollingHash(s):
    p, m = 31, 10**9 + 7
    hash_so_far = 0
    p_pow = 1
    for i in range(len(s)):
        hash_so_far = (hash_so_far + (1 + ord(s[i]) - ord('a')) * p_pow) % m
        p_pow = (p_pow * p) % m
    return hash_so_far

In [8]:
def hashJoin(dfs):
    # Getting the relation names
    keys = list(dfs.keys())
    # Iterating through relations
    for i in range(len(dfs)-1):
        h = defaultdict(list)
        # Hash phase
        if i == 0:
            table_1 = dfs[keys[i]].copy()
        else:
            table_1 = res_df.copy() 
            # Creating the hash table
        for ind,row in table_1.iterrows():
            h[rollingHash(row['OBJECT'])].append(list(row))
        # Join phase
            # Iterating through the second table and lookup in hash table and create new dataframe consisting of join of tables
        res_df = pd.DataFrame((list([*(s[:i+1]),r[1],r[1]]) for ind,r in dfs[keys[i+1]].iterrows() for s in h[rollingHash(r['SUBJECT'])])).rename(columns = {i+2:'OBJECT'})
    # Renaming the columns
    res_df = res_df.drop(len(dfs)-1,axis=1).rename(columns = {'OBJECT':keys[len(dfs)-1]})
    for i in range(len(dfs)-1):
        res_df.rename(columns={i: keys[i]}, inplace=True)
    return res_df

In [9]:
%%time
x = hashJoin(dfs)

Wall time: 2min 8s


In [10]:
x

,follows,friendOf,likes,hasReview
0,wsdbm:User630,wsdbm:User20,wsdbm:Product0,wsdbm:Review24
1,wsdbm:User26,wsdbm:User20,wsdbm:Product0,wsdbm:Review24
2,wsdbm:User197,wsdbm:User20,wsdbm:Product0,wsdbm:Review24
3,wsdbm:User351,wsdbm:User20,wsdbm:Product0,wsdbm:Review24
4,wsdbm:User396,wsdbm:User20,wsdbm:Product0,wsdbm:Review24
...,...,...,...,...
11415456,wsdbm:User186,wsdbm:User238,wsdbm:Product248,wsdbm:Review1248
11415457,wsdbm:User221,wsdbm:User238,wsdbm:Product248,wsdbm:Review1248
11415458,wsdbm:User361,wsdbm:User238,wsdbm:Product248,wsdbm:Review1248
11415459,wsdbm:User762,wsdbm:User238,wsdbm:Product248,wsdbm:Review1248


In [11]:
def sortMergeJoin(dfs):
    # Getting the relation names
    keys = list(dfs.keys())
    # Iterating through relations
    for i in range(len(dfs)-1):
        if i == 0:
            table_1 = dfs[keys[i]].copy()
        else:
            table_1 = res_df.copy()
        table_2 = dfs[keys[i+1]].copy()
        # Sort step
            # Sorting by keys of respective tables
        table_1.sort_values(by=['OBJECT'], inplace = True)
        table_2.sort_values(by=['SUBJECT'], inplace = True)
        # Merge Step
            # Initialise Row pointers for table 1 and 2 (tb1_p, tb2_p)
        tb1_p, tb2_p, rows = 0, 0, []
            # Iterating through rows in the tables
        while(tb1_p < len(table_1) and tb2_p < len(table_2)):
            # Get the current rows in the resp. tables
            vl1 = table_1.iloc[[tb1_p]]['OBJECT'].item()
            vl2 = table_2.iloc[[tb2_p]]['SUBJECT'].item()
            if vl1 == vl2:
                # Add the join of rows
                rows.append([*table_1.iloc[[tb1_p]].values.tolist()[0][:i+1], *[table_2.iloc[[tb2_p]]['OBJECT'].item()]*2])
                # Create temp row pointers
                temp_tb1_p = tb1_p + 1
                temp_tb2_p = tb2_p + 1
                # Cartesian product of row in first table with rows in second table
                while(temp_tb2_p < len(table_2)):
                    vl2_new = table_2.iloc[[temp_tb2_p]]['SUBJECT'].item()
                    if vl1 != vl2_new:
                        break
                    rows.append([*table_1.iloc[[tb1_p]].values.tolist()[0][:i+1], *[table_2.iloc[[temp_tb2_p]]['OBJECT'].item()]*2])
                    temp_tb2_p += 1
                # Cartesian product of row in second table with rows in first table
                while(temp_tb1_p < len(table_1)):
                    vl1_new = table_1.iloc[[temp_tb1_p]]['OBJECT'].item()
                    if vl1_new != vl2:
                        break
                    rows.append([*table_1.iloc[[temp_tb1_p]].values.tolist()[0][:i+1], *[table_2.iloc[[tb2_p]]['OBJECT'].item()]*2])
                    temp_tb1_p += 1
                # Update the row pointers
                tb1_p += 1
                tb2_p += 1
            elif vl1 < vl2:
                tb1_p += 1
            else:
                tb2_p +=1
        # Create new dataframe consisting of join of tables
        res_df = pd.DataFrame(rows).rename(columns = {i+2:'OBJECT'})
    # Renaming the columns
    res_df = res_df.drop(len(dfs)-1,axis=1).rename(columns = {'OBJECT':keys[len(dfs)-1]})
    for i in range(len(dfs)-1):
        res_df.rename(columns={i: keys[i]}, inplace=True)
    return res_df       

In [12]:
%%time
y = sortMergeJoin(dfs)

Wall time: 50min 14s


In [13]:
y

,follows,friendOf,likes,hasReview
0,wsdbm:User894,wsdbm:User152,wsdbm:Product0,wsdbm:Review24
1,wsdbm:User894,wsdbm:User152,wsdbm:Product0,wsdbm:Review1045
2,wsdbm:User894,wsdbm:User152,wsdbm:Product0,wsdbm:Review1074
3,wsdbm:User894,wsdbm:User152,wsdbm:Product0,wsdbm:Review1086
4,wsdbm:User894,wsdbm:User152,wsdbm:Product0,wsdbm:Review1107
...,...,...,...,...
11415456,wsdbm:User913,wsdbm:User413,wsdbm:Product9,wsdbm:Review1498
11415457,wsdbm:User512,wsdbm:User413,wsdbm:Product9,wsdbm:Review1498
11415458,wsdbm:User348,wsdbm:User114,wsdbm:Product9,wsdbm:Review1498
11415459,wsdbm:User853,wsdbm:User114,wsdbm:Product9,wsdbm:Review1498


PART C

WE USE PARTITIONING BEFORE JOINING WHICH CONSIDERABLY IMPROVES PERFORMANCE COMPARED TO SIMPLE SORT MERGE JOIN

In [8]:
def sortMergePartitionJoin(dfs):
    # Getting the relation names
    keys = list(dfs.keys())
    # Iterating through relations
    for i in range(len(dfs)-1):
        if i == 0:
            table_1 = dfs[keys[i]].copy()
        else:
            table_1 = res_df.copy()
        table_2 = dfs[keys[i+1]].copy()
        # Sort step
            # Sorting by keys of respective tables
        table_1.sort_values(by=['OBJECT'], inplace = True)
        table_2.sort_values(by=['SUBJECT'], inplace = True)
        #Partition step
            # Partition by key of respective relations
        table_1_groups = table_1.groupby('OBJECT')
        table_2_groups = table_2.groupby('SUBJECT')
        # Merge Step
        rows = []
            # Get keys of respective relations
        table_1_keys = table_1_groups.groups.keys()
        table_2_keys = table_2_groups.groups.keys()
        # Iterate through table 1 keys
        for key in table_1_keys:
            if key in table_2_keys:
                subset_1 = table_1_groups.get_group(key)
                subset_2 = table_2_groups.get_group(key)
                # Cartesion product
                rows += [[*(s[:i+1]),r[1],r[1]] for ind1,s in subset_1.iterrows() for ind2,r in subset_2.iterrows()]
         # Create new dataframe consisting of join of tables
        res_df = pd.DataFrame(rows).rename(columns = {i+2:'OBJECT'})
     # Renaming the columns
    res_df = res_df.drop(len(dfs)-1,axis=1).rename(columns = {'OBJECT':keys[len(dfs)-1]})
    for i in range(len(dfs)-1):
        res_df.rename(columns={i: keys[i]}, inplace=True)
    return res_df       

In [15]:
%%time
z = sortMergePartitionJoin(dfs)

Wall time: 10min 22s


In [16]:
z

,follows,friendOf,likes,hasReview
0,wsdbm:User178,wsdbm:User207,wsdbm:Product0,wsdbm:Review24
1,wsdbm:User178,wsdbm:User207,wsdbm:Product0,wsdbm:Review1045
2,wsdbm:User178,wsdbm:User207,wsdbm:Product0,wsdbm:Review1074
3,wsdbm:User178,wsdbm:User207,wsdbm:Product0,wsdbm:Review1086
4,wsdbm:User178,wsdbm:User207,wsdbm:Product0,wsdbm:Review1107
...,...,...,...,...
11415456,wsdbm:User293,wsdbm:User598,wsdbm:Product9,wsdbm:Review1216
11415457,wsdbm:User293,wsdbm:User598,wsdbm:Product9,wsdbm:Review1043
11415458,wsdbm:User293,wsdbm:User598,wsdbm:Product9,wsdbm:Review865
11415459,wsdbm:User293,wsdbm:User598,wsdbm:Product9,wsdbm:Review661


In [17]:
%%time
# Compared to built in pandas merge function
dfs['follows'].merge(dfs['friendOf'], left_on = 'OBJECT', right_on = 'SUBJECT').rename(columns = {'OBJECT_y':'OBJECT'})\
              .merge(dfs['likes'], left_on = 'OBJECT', right_on = 'SUBJECT').rename(columns = {'OBJECT_y':'OBJECT'})\
              .merge(dfs['hasReview'], left_on = 'OBJECT', right_on = 'SUBJECT')

C:\Users\sajus\anaconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'OBJECT_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(
C:\Users\sajus\anaconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'OBJECT_x', 'SUBJECT_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


Wall time: 1.01 s


,SUBJECT_x,OBJECT_x,SUBJECT_y,OBJECT_x,SUBJECT_x,OBJECT_x,SUBJECT_y,OBJECT_y
0,wsdbm:User0,wsdbm:User27,wsdbm:User27,wsdbm:User25,wsdbm:User25,wsdbm:Product163,wsdbm:Product163,wsdbm:Review6
1,wsdbm:User0,wsdbm:User27,wsdbm:User27,wsdbm:User25,wsdbm:User25,wsdbm:Product163,wsdbm:Product163,wsdbm:Review66
2,wsdbm:User0,wsdbm:User27,wsdbm:User27,wsdbm:User25,wsdbm:User25,wsdbm:Product163,wsdbm:Product163,wsdbm:Review203
3,wsdbm:User0,wsdbm:User27,wsdbm:User27,wsdbm:User25,wsdbm:User25,wsdbm:Product163,wsdbm:Product163,wsdbm:Review210
4,wsdbm:User0,wsdbm:User27,wsdbm:User27,wsdbm:User25,wsdbm:User25,wsdbm:Product163,wsdbm:Product163,wsdbm:Review212
...,...,...,...,...,...,...,...,...
11415456,wsdbm:User914,wsdbm:User310,wsdbm:User310,wsdbm:User728,wsdbm:User728,wsdbm:Product192,wsdbm:Product192,wsdbm:Review1192
11415457,wsdbm:User914,wsdbm:User310,wsdbm:User310,wsdbm:User728,wsdbm:User728,wsdbm:Product192,wsdbm:Product192,wsdbm:Review1294
11415458,wsdbm:User914,wsdbm:User310,wsdbm:User310,wsdbm:User728,wsdbm:User728,wsdbm:Product192,wsdbm:Product192,wsdbm:Review1325
11415459,wsdbm:User914,wsdbm:User310,wsdbm:User310,wsdbm:User728,wsdbm:User728,wsdbm:Product192,wsdbm:Product192,wsdbm:Review1373


 Running for the other dataset

In [3]:
df2 = pd.read_csv("watdiv.10M.nt", header=None)

In [4]:
df2 = df2[0].str.split('\t', expand=True).rename(columns = {0:'SUBJECT',1:'ATTRIBUTE',2:'OBJECT'})

In [5]:
df2['OBJECT'] = df2['OBJECT'].str[:-2]

In [6]:
groups2 = df2.groupby('ATTRIBUTE')[['SUBJECT','OBJECT']]

In [7]:
dfs2 = {}
for attribute in ['follows','friendOf', 'likes', 'hasReview']:
    dfs2[attribute] =  groups2.get_group(df2[df2['ATTRIBUTE'].str.contains(attribute)]['ATTRIBUTE'].unique()[0])

In [ ]:
x2 = sortMergePartitionJoin(dfs2)